In [1]:
from langchain.chains.llm import LLMChain
from langchain.chains.sequential import SequentialChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

import os
from dotenv import load_dotenv

load_dotenv('../.env')

model_name = os.getenv("LLM_MODEL") or "gpt-4o-mini"
model_provider = os.getenv("LLM_MODEL_PROVIDER") or "openai"

In [2]:
llm = ChatOpenAI(model=model_name, temperature=0.7)

In [4]:
from langchain_core.prompts import PromptTemplate

prompt1 = PromptTemplate.from_template(
    "다음 식당 리뷰를 한 문장으로 요약하세요.\n\n{review}"
)

chain1 = LLMChain(llm=llm, prompt=prompt1, output_key="summary")

prompt2 = PromptTemplate.from_template(
    "다음 식당 리뷰를 읽고 0점부터 10점 사이에서 긍정/부정 점수를 매기세요. 숫자로만 대답하세요. {review}"
)

chain2 = LLMChain(llm=llm, prompt=prompt2, output_key="sentiment_score")

prompt3 = PromptTemplate.from_template(
    "다음 식당 리뷰 요약에 대해 공손한 답변을 작성하세요.\n리뷰 요약:{summary}"
)

chain3 = LLMChain(llm=llm, prompt=prompt3, output_key="reply")

In [5]:
# 체인 설정
all_chain = SequentialChain(
    chains=[chain1, chain2, chain3],
    input_variables=["review"],
    output_variables=["summary", "sentiment_score", "reply"],
    verbose=True
)


In [6]:
# 식당 리뷰 입력
review = """
이 식당은 맛도 좋고 분위기도 좋았습니다. 가격 대비 만족도가 높아요.
하지만, 서비스 속도가 너무 느려서 조금 실망스러웠습니다.
전반적으로는 다시 방문할 의사가 있습니다.
"""

In [7]:
# 체인 실행 및 결과 출력
try:
    result = all_chain.invoke(input={"review": review})
    print(f'summary 결과 \n{result["summary"]}\n')
    print(f'sentiment_score 결과 \n{result["sentiment_score"]}\n')
    print(f'reply 결과 \n{result["reply"]}\n')
except Exception as e:
    print(f"오류 발생: {e}")



> Entering new SequentialChain chain...

> Finished chain.
summary 결과 
맛과 분위기는 좋았지만 서비스 속도가 느려 아쉬웠지만, 전반적으로 재방문 의사가 있는 식당입니다.

sentiment_score 결과 
7

reply 결과 
고객님께서 저희 식당의 맛과 분위기를 만족해 주시고, 재방문 의사까지 보여주셔서 진심으로 감사드립니다. 다만, 서비스 속도에 있어 불편을 드린 점 정말 죄송하게 생각합니다. 앞으로 더욱 신속하고 친절한 서비스를 제공할 수 있도록 노력하겠습니다. 소중한 의견에 다시 한 번 감사드리며, 다음 방문 때는 더욱 만족하실 수 있도록 최선을 다하겠습니다.

